<a href="https://colab.research.google.com/github/xpandalord/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/module2-regression-2/7_7_LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
# taking a look at the dataset
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
# checking what type is the created column
df['created'].describe()

count                   48817
unique                  48148
top       2016-04-15 02:24:25
freq                        3
Name: created, dtype: object

In [5]:
# changing the type of created to datetime
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)

In [6]:
# creating a column that totals bedrooms and bathrooms
df['total_rooms'] = df['bedrooms'] + df['bathrooms']

In [7]:
# creating a column that checks if cats or dogs are allowed
# 1 means one but not the other is allowed and 2 means both are allowed
df['cats_or_dogs_allowed'] = df['cats_allowed'] + df['dogs_allowed']

In [8]:
# setting aside the prices
y = df['price']
y.shape

(48817,)

In [9]:
# dropping the prices
X = df.drop('price', axis = 1)
X.shape

(48817, 35)

In [10]:
# splitting the training data and testing data into before june
# and after june
X_train = X[X['created'].dt.month < 6]
y_train = y[y.index.isin(X_train.index)]
X_test = X[X['created'].dt.month >= 6]
y_test = y[y.index.isin(X_test.index)]

In [11]:
# get mean baseline
y_train.mean()

3575.604007034292

In [12]:
# Train Error
from sklearn.metrics import mean_absolute_error

y_pred = [y_train.mean()] * len(y_train)

In [13]:
# calculating MAE
mae = mean_absolute_error(y_train, y_pred)
print(f'Train Error (NYC Apartment Prices): ${mae:.2f}')

Train Error (NYC Apartment Prices): $1201.88


In [14]:
# Test Error
y_pred = [y_train.mean()] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error (NYC Apartment Prices: ${mae:.2f}')

Test Error (NYC Apartment Prices: $1197.71


In [15]:
X_train['total_rooms']

2        2.0
3        2.0
4        5.0
5        6.0
6        3.0
        ... 
49346    2.0
49348    2.0
49349    2.0
49350    1.0
49351    3.0
Name: total_rooms, Length: 31844, dtype: float64

In [16]:
y_train

2        2850
3        3275
4        3350
5        7995
6        3600
         ... 
49346    4500
49348    3950
49349    2595
49350    3350
49351    2200
Name: price, Length: 31844, dtype: int64

In [17]:
# For Graphing using plotly
import pandas as pd
import plotly.express as px

px.scatter(
    X_train,
    x='total_rooms',
    y=y_train,
    trendline='ols', # Ordinary Least Squares
)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [18]:
# 1. Import the appropriate estimator class from Scikit-Learn
from sklearn.linear_model import LinearRegression

In [19]:
# 2. Instantiate this class
model = LinearRegression()

In [20]:
# 4. Fit the model
model.fit(X_train[['total_rooms']], y_train)
y_pred = model.predict(X_train[['total_rooms']])
mae - mean_absolute_error(y_train, y_pred)
print(f'Train Error: ${mae:.2f}')

Train Error: $1197.71


In [21]:
# 5. Apply the model to new data
y_pred = model.predict(X_test[['total_rooms']])
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:.2f}')

Test Error: $910.51


In [22]:
# Creating a 3d scatter plot using two features and price
px.scatter_3d(
    X_train,
    x='total_rooms',
    y='cats_or_dogs_allowed',
    z=y_train,
    title='Prices Based on # of Rooms and Pet Allowance'
)

In [23]:
# Re-arrage X features matrics
features = ['total_rooms', 'cats_or_dogs_allowed']
print(f'Linear Regression, dependent on: {features}')

Linear Regression, dependent on: ['total_rooms', 'cats_or_dogs_allowed']


In [27]:
# Fit the model
lr = LinearRegression()
lr.fit(X_train[features], y_train)

mae = mean_absolute_error(y_train, lr.predict(X_train[features]))
print(f'Training error: ${mae:.2f}')

Training error: $888.28


In [30]:
# Apply the model to new data
mae = mean_absolute_error(y_test, lr.predict(X_test[features]))
print(f'Testing error: ${mae:.2f}')

Testing error: $905.28


In [36]:
# the intercept and coefficients for the regression
lr.intercept_, lr.coef_

(1270.6609889046963, array([809.54448582, 100.97408923]))

In [38]:
# formatting the regression equation
beta0 = lr.intercept_
beta1, beta2 = lr.coef_
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')

y = 1270.6609889046963 + 809.5444858225759x1 + 100.97408922536198x2


In [39]:
# This is easier to read
print('Intercept', lr.intercept_)
coefficients = pd.Series(lr.coef_, features)
print(coefficients.to_string())

Intercept 1270.6609889046963
total_rooms             809.544486
cats_or_dogs_allowed    100.974089


In [40]:
# doing some tests to check
lr.predict([[5,1]])

array([5419.35750724])

In [43]:
# changing the pet perk
lr.predict([[5,0]])

array([5318.38341802])

In [42]:
# changing the number of rooms
lr.predict([[3,1]])

array([3800.2685356])

In [67]:
# Get regression metrics RMSE, MAE, and  R2
# for both the train and test data.
import numpy as np
from sklearn.metrics import *

def squared_errors(df, features, target, coefficients):
  """
  Visualize linear regression, with squared errors,
  in 3D: 2 features + 1 target.

  Use the beta0, beta1, and beta2 parameters to "fit the model" manually

  df : Pandas DataFrame
  features : array, features 1 and 2 columns in df
  target : string, feature target column
  coefficients : array, intercept and coefficients for polynomial equation
  """
  # Make predictions
  x = features[0]
  y = features[1]
  z = target
  z_pred = coefficients[0] + coefficients[1]*df[x] + coefficients[2]*df[y]

  # Print regression metrics
  mse = mean_squared_error(z, z_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(z, z_pred)
  r2 = r2_score(z, z_pred)
  print(f'Mean Squared Error: ${mse:.2f}')
  print(f'Root Mean Squared Error: ${rmse:.2f}')
  print(f'Mean Absolute Error: ${mae:.2f}')
  print(f'R^2: ${r2:.2f}')

In [68]:
# testing some coefficients
features = ['total_rooms', 'cats_or_dogs_allowed']
coefficients = [1271, 810, 101]
squared_errors(X_train, features, y_train, coefficients)

Mean Squared Error: $1788230.02
Root Mean Squared Error: $1337.25
Mean Absolute Error: $888.56
R^2: $0.42
